In [1]:
from utils import *

In [39]:
pdxnet_sheet = '/Users/tushar/Downloads/PDXNetAllModels (4).tsv'
pdxnet_df = read_metadata_with_fields(pdxnet_sheet)
wustl_df = pdxnet_df[pdxnet_df['Contributor'] == 'WUSTL'].reset_index(drop=True)
wustl_df['model_id'] = wustl_df['ContributorPDX.ID'].str.split('; ')
wustl_df = wustl_df.explode('model_id').reset_index(drop=True)
wustl_df['model_id'] = wustl_df['model_id'].str.split(', ')
wustl_df = wustl_df.explode('model_id').reset_index(drop=True)
wustl_df['model_id'] = wustl_df['model_id'].str.split(';')
wustl_df = wustl_df.explode('model_id').reset_index(drop=True)
wustl_df['model_id'] = wustl_df['model_id'].str.split(',')
wustl_df = wustl_df.explode('model_id').reset_index(drop=True)
wustl_df['model_id'] = [ f[0] for f in wustl_df['model_id'].str.split(' ')]

In [40]:
wustl_df[wustl_df['model_id'].duplicated(keep=False)]

,PDXSource,Contributor,ContributorPDX.ID,PDMR.Patient.ID,Gender,CTEP.SDCCode,CTEP.SDCDescription,Diagnosis.Subtype,Disease.Body.Location,Age.at.Diagnosis,...,Grade.Stage.Information,Patient.Notes,Molecular.and.IHC.Data,Has.Smoked.100.Cigarettes,Race,Ethnicity,Additional.Medical.History,OncoTree.Code,Submission,model_id
9,PDXNet Consortium Members,WUSTL,WU-0010-T1 (001-R1),K20605,Male,10038045,Adenocarcinoma - rectum,-,Digestive/Gastrointestinal,60,...,TNM,"Tumor Grade/Stage: T3N1bM1b, moderately differ...",-,Not Provided,White,Not Hispanic or Latino,-,NaN,2/26/2019,WU-0010-T1
184,PDXNet Consortium Members,WUSTL,WU-0010-T1,K96138,Male,10038045,Adenocarcinoma - rectum,mucinous,Digestive/Gastrointestinal,60,...,TNM (Pathological),"Tumor Grade/Stage: Moderately differentiated, ...",-,Yes,Not Provided,Not Hispanic or Latino,"Family History: 1st Degree Relative, Skin cancer",NOT_SELECTED,1/5/2023,WU-0010-T1


In [41]:
wustl_df.columns

Index(['PDXSource', 'Contributor', 'ContributorPDX.ID', 'PDMR.Patient.ID',
       'Gender', 'CTEP.SDCCode', 'CTEP.SDCDescription', 'Diagnosis.Subtype',
       'Disease.Body.Location', 'Age.at.Diagnosis', 'Date.of.Diagnosis',
       'Has.Known.Metastatic.Disease', 'Grade.Stage.Information',
       'Patient.Notes', 'Molecular.and.IHC.Data', 'Has.Smoked.100.Cigarettes',
       'Race', 'Ethnicity', 'Additional.Medical.History', 'OncoTree.Code',
       'Submission', 'model_id'],
      dtype='object')

In [48]:
wustl_df.to_csv('/Users/tushar/Downloads/PDXNetAllModels_models.tsv', sep='\t', index=False)

In [52]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

In [79]:
url = 'https://pdx.wustl.edu/pdx/'
response = get(url)
soup = BeautifulSoup(response.text, 'html.parser')
ctypes_table = soup.find_all('table' , {'id': 'ctypes'})[0]
data = []
for row in ctypes_table.find_all('tr')[1:-1]:  # skip header and total row
    cells = row.find_all('td')
    cancer_type = cells[0].get_text(strip=True)
    link = cells[0].find('a')['href']
    description = cells[1].get_text(strip=True)
    number_of_models = int(cells[2].get_text(strip=True))
    data.append({
        'type': cancer_type,
        'link': link,
        'description': description,
        'models': number_of_models
    })

In [137]:
from tqdm import tqdm

In [138]:
final_df = pd.DataFrame()
for d in tqdm(data):
    new_url = url + d['link']
    response = get(new_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'center'})

    # Extract headers
    headers = [th.get_text(strip=True).replace('\n', ' ') for th in table.find_all('th')]

    # Extract rows
    rows = []
    for tr in table.find_all('tr')[1:]:
        tds = tr.find_all('td')
        row = []
        for i, td in enumerate(tds):
            if i == 8:  # PDX Samples
                # Join all span text with commas or line breaks
                samples = [span.get_text(strip=True) for span in td.find_all('span')]
                row.append(", ".join(samples))
            elif i == 9:  # Genetic Alterations (might be empty or have links/text)
                links = [a['href'] for a in td.find_all('a', href=True)]
                row.append(", ".join(links))
            else:
                row.append(td.get_text(strip=True))
        if row:
            rows.append(row)

    # Create DataFrame
    df = pd.DataFrame(rows, columns=headers)
    df['type'] = d['type']
    final_df = pd.concat([final_df, df])
final_df

 15%|█▌        | 4/26 [00:04<00:25,  1.14s/it]


KeyboardInterrupt: 

In [92]:
wustl_df_mol_data = final_df[final_df['Source'] == 'WUSTL'].reset_index(drop=True)
wustl_df_mol_data = wustl_df_mol_data[['ModelName', 'Source', 'Patient', 'Patient Race(Self-reported)', 'PatientGender', 'Specimen TreatmentPrior to Collection', 'PDX Samples', 'GeneticAlterations']]
wustl_df_mol_data.to_csv('wustl_mol_data.tsv', sep='\t', index=False)

In [140]:
mutation_template = read_metadata_with_fields(join(templates, 'mut/mutation_template_external-mutation_template_external.tsv'))
cna_template = read_metadata_with_fields(join(templates, 'cna/cna_template-sheet.tsv'))
mms = read_metadata_with_fields(join(home, 'WUSTL/WUSTL_molecular_metadata-sample.tsv'))

In [131]:
mms.columns

Index(['Field', 'model_id', 'sample_id', 'sample_origin', 'passage',
       'host_strain_name', 'host_strain_nomenclature',
       'engrafted_tumor_collection_site', 'raw_data_url', 'platform_id'],
      dtype='object')

In [141]:
final_mut_df = pd.DataFrame()

for _, row in tqdm(wustl_df_mol_data.iterrows()):
    model_id = row['ModelName']
    sample_ids = row['PDX Samples'].split(', ')
    links = row['GeneticAlterations'].split(', ')
    sid_link_dict = dict(zip(sample_ids, links))
    for sid in sid_link_dict:
        new_url = url + sid_link_dict[sid]
        response = get(new_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find_all('table')[1]
        # Extract headers
        headers = [th.get_text(strip=True).replace('\n', ' ') for th in table.find_all('th')]

        # Extract rows
        rows = []
        for tr in soup.find_all('tr')[1:]:
            cells = tr.find_all('td')
            row = [td.get_text(strip=True).replace('\n', '') for td in cells]
            if row:
                rows.append(row)

        # Create DataFrame

        df = pd.DataFrame(rows, columns=headers).iloc[3:]

        temp_cna = df[df['CN status']!= '-']
        cna_df = cna_template
        cna_df['symbol'] = temp_cna['Gene']
        cna_df['copy_number_status'] = temp_cna['CN status']
        cna_df['gistic_value'] = temp_cna['SomaticCopy Number']
        cna_df['sample_id'] = sid
        cna_df['platform_id'] = 'wustl_cna_status'

        temp_mut = df[df['SomaticMutations']!= '-']
        if temp_mut.shape[0] > 0:
            mut_df = mutation_template
            mut_df['symbol'] = temp_mut['Gene']
            mut_df['amino_acid_change'] = temp_mut['SomaticMutations']
            mut_df['allele_frequency'] = temp_mut['VAF']
            mut_df['consequence'] = temp_mut['Classification']
            mut_df['variant_class'] = temp_mut['Type']
            mut_df['sample_id'] = sid
            mut_df['platform_id'] = 'wustl_mutation'
            final_mut_df = pd.concat([final_mut_df, mut_df])

            new_row_df = pd.DataFrame([['', model_id, sid, 'xenograft', '0', 'NSG', 'NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</sup>/SzJ', '', '', 'wustl_mutation']], columns=mms.columns)
            mms = pd.concat([mms, new_row_df], ignore_index=True)

        #cna_df.to_csv(f'cna/WUSTL_cna_{sid}.tsv', sep='\t', index=False)
        #new_row_df = pd.DataFrame([['', model_id, sid, 'xenograft', '0', 'NSG', 'NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</sup>/SzJ', '', '', 'wustl_cna_status']], columns=mms.columns)
        #mms = pd.concat([mms, new_row_df], ignore_index=True)
mms.to_csv(join(home, 'WUSTL/WUSTL_molecular_metadata-sample_2.tsv'), sep='\t', index=False)

133it [21:20,  9.62s/it]


In [171]:
final_mut_df.to_csv('WUSLT_mut.tsv', sep='\t', index=False)

In [190]:
final_mut_df.head()

,sample_id,symbol,read_depth,allele_frequency,chromosome,strand,seq_start_position,seq_end_position,ref_allele,alt_allele,variation_id,platform,amino_acid_change,consequence,variant_class,platform_id
281,WU-0029-T1-X1P0,ADAMTS7,NaN,0.513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.R1635C,Missense,SNP,wustl_mutation
284,WU-0029-T1-X1P0,ADAMTSL1,NaN,0.472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.E238*,Nonsense,SNP,wustl_mutation
329,WU-0029-T1-X1P0,ADRA2C,NaN,0.733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.E410K,Missense,SNP,wustl_mutation
693,WU-0029-T1-X1P0,ARID1A,NaN,0.682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Q557*,Nonsense,SNP,wustl_mutation
870,WU-0029-T1-X1P0,BAG5,NaN,0.077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.E419K,Missense,SNP,wustl_mutation


In [196]:
aa_change_df = final_mut_df.drop_duplicates(['symbol', 'amino_acid_change'])[['symbol', 'amino_acid_change']]

In [197]:
aa_change_df

,symbol,amino_acid_change
281,ADAMTS7,p.R1635C
284,ADAMTSL1,p.E238*
329,ADRA2C,p.E410K
693,ARID1A,p.Q557*
870,BAG5,p.E419K
...,...,...
3108,TPP1,p.H166Mfs*17
2619,SMPDL3A,p.T51S
329,APC,p.K1310Dfs*4p.R213*
3108,GPR37,p.R482W


In [233]:
import requests
import csv

# Input: list of (Gene Symbol, Amino Acid Change)
variants = [
    ("BRAF", "p.V600E"),
    ("TP53", "p.R175H"),
    ("CDH1", "p.R598Q")
    # add more here
]

vcf_lines = [
    "##fileformat=VCFv4.2",
    "#CHROM\tPOS\tSTOP\tID\tREF\tALT\tcdna\taa_change\tgene\tQUAL\tFILTER\tINFO"
]

def get_hgnc_symbol(symbol):
    url = "https://mygene.info/v3/query"
    params = {
        "q": symbol,
        "species": "human",
        "fields": "symbol,name,HGNC",
        "size": 2
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data["hits"]:
        if len(data["hits"]) > 1:
            return [data["hits"][0]["symbol"], data["hits"][1]["symbol"]]
        else:
            return [data["hits"][0]["symbol"], '']
    else:
        return None

def get_clinvar_variants(gene, aa_change):
    search_term = f"{gene}[gene]+{aa_change}[varname]"
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "clinvar",
        "term": search_term,
        "retmode": "json",
        "retmax": "1",
        "retmode": "json",
        "rettype": "clinvarset"
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    ids = data.get("esearchresult", {}).get("idlist", [])
    return ids

def fetch_clinvar_record(variant_id):
    summary_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    params = {
        "db": "clinvar",
        "id": variant_id,
        "retmode": "json"
    }

    resp = requests.get(summary_url, params=params)
    resp.raise_for_status()
    record = resp.json()["result"][variant_id]
    return record

def parse_cdna_change(cdna_change):
    import re
    match = re.match(r"c\.(\d+)([ACGT])>([ACGT])", cdna_change)
    if match:
        ref = match.group(2)
        alt = match.group(3)
        return ref, alt
    else:
        raise ValueError(f"Invalid cDNA change format: {cdna_change}")



def extract_vcf_info(record, gene, aa_change):
    if "variation_set" in record:
        var = record["variation_set"][0]
        cdna_change = var['cdna_change']
        for placement in var.get("variation_loc", []):
            if placement.get("assembly_name") == "GRCh38":
                    chr = placement['chr']
                    start_pos = placement["start"]
                    end_pos = placement["stop"]
                    ref, alt = parse_cdna_change(cdna_change)
                    return chr, start_pos, end_pos,".", ref, alt, cdna_change, gene, aa_change
    return None

# Loop through input variants
for _, row in aa_change_df.iterrows():
    gene, aa_change = row
    print(f"Looking up {gene} {aa_change}...")
    ids = get_clinvar_variants(gene, aa_change)
    if not ids:
        hits = get_hgnc_symbol(gene)
        if hits is not None:
            ids = get_clinvar_variants(hits[0], aa_change)
            if (not ids or len(ids) == 0) and hits[1]!= '':
                ids = get_clinvar_variants(hits[1], aa_change)
                if not ids or len(ids) == 0:
                    print(f"❌ No ClinVar match for {gene} {aa_change}")
                    continue
    if len(ids) == 0:
        continue
    record = fetch_clinvar_record(ids[0])
    #print(record)
    vcf_info = extract_vcf_info(record, gene, aa_change)
    print(vcf_info)
    if vcf_info:
        vcf_lines.append("\t".join(map(str, list(vcf_info) + [".", ".", "."])))
    else:
        print(f"⚠️ Could not extract VCF info for {gene} {aa_change}")

# Write to VCF file
with open("clinvar_variants.vcf", "w") as f:
    f.write("\n".join(vcf_lines))

print("✅ VCF written to clinvar_variants.vcf")


❌ No ClinVar match for ADAMTS7 p.R1635C
❌ No ClinVar match for ARID1A p.Q557*
❌ No ClinVar match for BRINP3 p.K600T
❌ No ClinVar match for CDC42EP1 p.A256T
('16', '68822082', '68822082', '.', 'G', 'A', 'c.1793G>A', 'CDH1', 'p.R598Q')
❌ No ClinVar match for CENPB p.S188L
❌ No ClinVar match for DSCAM p.E1093K
❌ No ClinVar match for ERBB3 p.S1234F
('20', '6077214', '6077214', '.', 'G', 'A', 'c.1993G>A', 'FERMT1', 'p.D665N')
❌ No ClinVar match for GAR1 p.R13Q
❌ No ClinVar match for GK2 p.N277Tfs*25
❌ No ClinVar match for GOLGA5 p.Q618*
❌ No ClinVar match for GPR32 p.R355Gfs*16
❌ No ClinVar match for IGF2R p.E2412K
('7', '16308536', '16308536', '.', 'C', 'T', 'c.776C>T', 'ISPD', 'p.P259L')
❌ No ClinVar match for KIAA1586 p.E378K
❌ No ClinVar match for MORC1 p.Q857H
❌ No ClinVar match for NCOR1 p.S174L
❌ No ClinVar match for RGS4 p.R269Q
('4', '989096', '989096', '.', 'G', 'A', 'c.1843G>A', 'SLC26A1', 'p.D615N')
❌ No ClinVar match for SP110 p.D686Y
❌ No ClinVar match for T p.R169*
❌ No ClinV

In [245]:
clinvar_vcf = pd.DataFrame([r.split('\t') for r in vcf_lines[2:]], columns=[vcf_lines[1].split('\t')])
clinvar_vcf

,#CHROM,POS,STOP,ID,REF,ALT,cdna,aa_change,gene,QUAL,FILTER,INFO
0,16,68822082,68822082,.,G,A,c.1793G>A,CDH1,p.R598Q,.,.,.
1,20,6077214,6077214,.,G,A,c.1993G>A,FERMT1,p.D665N,.,.,.
2,7,16308536,16308536,.,C,T,c.776C>T,ISPD,p.P259L,.,.,.
3,4,989096,989096,.,G,A,c.1843G>A,SLC26A1,p.D615N,.,.,.
4,17,7674252,7674252,.,G,C,c.711G>C,TP53,p.M237I,.,.,.
5,6,52982621,52982621,.,C,G,c.499C>G,GSTA4,.,.,.,.
6,11,103286310,103286310,.,G,A,c.10946G>A,DYNC2H1,p.R3656H,.,.,.
7,22,36312212,36312212,.,C,T,c.1565C>T,MYH9,p.P522L,.,.,.
8,3,36899201,36899201,.,G,A,c.341G>A,TRANK1,p.R70H,.,.,.
9,19,49104729,49104729,.,C,T,c.571C>T,SNRNP70,p.R191W,.,.,.


In [267]:
final_mut_df.reset_index(drop=True).merge(clinvar_vcf.reset_index(drop=True), left_on="symbol", right_on="aa_change")

ValueError: The column label 'aa_change' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.

In [262]:
final_mut_df[final_mut_df['symbol'] == 'GSTA4']

,sample_id,symbol,read_depth,allele_frequency,chromosome,strand,seq_start_position,seq_end_position,ref_allele,alt_allele,variation_id,platform,amino_acid_change,consequence,variant_class,platform_id
4090,WU-L012-T1-X1P0,GSTA4,NaN,0.312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.,Splice_Site,DEL,wustl_mutation


In [261]:
clinvar_vcf['gene']

,gene
0,p.R598Q
1,p.D665N
2,p.P259L
3,p.D615N
4,p.M237I
5,.
6,p.R3656H
7,p.P522L
8,p.R70H
9,p.R191W


In [223]:
import requests

def get_hgnc_symbol(symbol):
    url = "https://mygene.info/v3/query"
    params = {
        "q": symbol,
        "species": "human",
        "fields": "symbol,name,HGNC",
        "size": 2
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data["hits"]:
        return data['hits']
    else:
        return None

# Example usage
data = get_hgnc_symbol("Tp53")   # returns "TP53"
#print(get_hgnc_symbol("P53"))    # returns "TP53" (aliases work too)


In [226]:
data[1]

{'HGNC': '44657',
 '_id': '729264',
 '_score': 63.653282,
 'name': 'TP53 target 3D',
 'symbol': 'TP53TG3D'}

In [179]:
record = fetch_clinvar_record('188221')

In [202]:
record['variation_set'][0]

{'measure_id': '27621',
 'variation_xrefs': [{'db_source': 'ClinGen', 'db_id': 'CA122538'},
  {'db_source': 'UniProtKB', 'db_id': 'P01116#VAR_016026'},
  {'db_source': 'OMIM', 'db_id': '190070.0005'},
  {'db_source': 'OMIM', 'db_id': '190070.0025'},
  {'db_source': 'dbSNP', 'db_id': '121913529'}],
 'variation_name': 'NM_004985.5(KRAS):c.35G>A (p.Gly12Asp)',
 'cdna_change': 'c.35G>A',
 'aliases': [],
 'variation_loc': [{'status': 'current',
   'assembly_name': 'GRCh38',
   'chr': '12',
   'band': '12p12.1',
   'start': '25245350',
   'stop': '25245350',
   'inner_start': '',
   'inner_stop': '',
   'outer_start': '',
   'outer_stop': '',
   'display_start': '25245350',
   'display_stop': '25245350',
   'assembly_acc_ver': 'GCF_000001405.38',
   'annotation_release': '',
   'alt': '',
   'ref': ''},
  {'status': 'previous',
   'assembly_name': 'GRCh37',
   'chr': '12',
   'band': '12p12.1',
   'start': '25398284',
   'stop': '25398284',
   'inner_start': '',
   'inner_stop': '',
   'oute

In [181]:
extract_vcf_info(record)

In [132]:
mms

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,NaN,NaN
3,#Essential?,essential,essential,essential,essential,essential,essential,desirable,desirable,essential
4,NaN,WUSTL WHIM5,WHIM5,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
5,NaN,WUSTL WHIM6,WHIM6,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
6,NaN,WUSTL WHIM8,WHIM8,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
7,NaN,WUSTL WHIM9,WHIM9,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
8,NaN,WUSTL WHIM11,WHIM11,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
9,NaN,WUSTL WHIM12,WHIM12,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina


In [136]:
mms

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,NaN,NaN
3,#Essential?,essential,essential,essential,essential,essential,essential,desirable,desirable,essential
4,NaN,WUSTL WHIM5,WHIM5,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
5,NaN,WUSTL WHIM6,WHIM6,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
6,NaN,WUSTL WHIM8,WHIM8,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
7,NaN,WUSTL WHIM9,WHIM9,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
8,NaN,WUSTL WHIM11,WHIM11,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina
9,NaN,WUSTL WHIM12,WHIM12,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,mammary fat pad,"phs000611, GSE46604, PRJNA217715",mut_wgs_illumina


In [120]:
len(table)

2

In [124]:
table[0]

<table class="center"><thead><tr class="gray-bg"><th class="calign">Variant Type</th><th class="calign">Calling Status</th></tr></thead><tbody><tr><td>Mutations</td><td>Called</td></tr><tr><td>Copy number</td><td>Called</td></tr><tr><td>Fusions</td><td>Called</td></tr></tbody></table>

In [122]:
# Extract headers
headers = [th.get_text(strip=True).replace('\n', ' ') for th in table[1].find_all('th')]

# Extract rows
rows = []
for tr in soup.find_all('tr')[1:]:
    cells = tr.find_all('td')
    row = [td.get_text(strip=True).replace('\n', '') for td in cells]
    if row:
        rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows, columns=headers)


In [123]:
df

,Gene,SomaticMutations,Classification,Type,VAF,SomaticCopy Number,CN status,Fusions
0,Mutations,Called,None,None,None,None,None,None
1,Copy number,Called,None,None,None,None,None,None
2,Fusions,Called,None,None,None,None,None,None
3,A2M,-,-,-,-,5,Amp,-
4,A2ML1,-,-,-,-,5,Amp,-
...,...,...,...,...,...,...,...,...
9157,ZSWIM6,-,-,-,-,1,Loss,-
9158,ZSWIM7,-,-,-,-,3,Gain,-
9159,ZW10,-,-,-,-,1,Loss,-
9160,ZXDC,-,-,-,-,1,Loss,-


In [151]:
from utils import read_metadata_without_fields
metadata = pd.DataFrame()
for provider in get_dirs(home):
    new_path = join(home, provider)
    tsv_files = sorted([join(new_path, f) for f in get_files(new_path) if f.endswith('.tsv') and not f.__contains__('image') and not f.__contains__('molecular_metadata')])
    if f'{home}{provider}/{provider}_metadata-pdx_model.tsv' in tsv_files:
        df = read_metadata_without_fields(f'{home}{provider}/{provider}_metadata-pdx_model.tsv')
        f =  f'{home}{provider}/{provider}_metadata-model_validation.tsv'
        df = df.merge(read_metadata_without_fields(f), on='model_id', how='left')
        f =  f'{home}{provider}/{provider}_metadata-patient_sample.tsv'
        df = df.merge(read_metadata_without_fields(f), on='model_id', how='left')
        f =  f'{home}{provider}/{provider}_metadata-patient.tsv'
        df = df.merge(read_metadata_without_fields(f), on='patient_id', how='left')
    metadata = pd.concat([metadata, df], ignore_index=True)

metadata


,model_id,host_strain_name,host_strain_nomenclature,engraftment_site,engraftment_type,sample_type,sample_state,passage_number,parent_id,origin_patient_sample_id,...,Unnamed: 1015,Unnamed: 1016,Unnamed: 1017,Unnamed: 1018,Unnamed: 1019,Unnamed: 1020,Unnamed: 1021,Unnamed: 1022,Unnamed: 1023,Unnamed: 1024
0,VHIO_PC-PDX-1,NOD SCID,NOD.CB-17-Prkdc <sup>scid</sup>/Rj,flank,heterotopic,tissue fragment,cryopreserved,"0,1,2,3,4,5",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VHIO_PC-PDX-3,NOD SCID,NOD.CB-17-Prkdc <sup>scid</sup>/Rj,flank,heterotopic,tissue fragment,cryopreserved,"1,2,3",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VHIO_PC-PDX-6,NOD SCID,NOD.CB-17-Prkdc <sup>scid</sup>/Rj,flank,heterotopic,tissue fragment,cryopreserved,"0,1,2,3,4,5,6,7",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VHIO_PC-PDX-9,NOD SCID,NOD.CB-17-Prkdc <sup>scid</sup>/Rj,flank,heterotopic,tissue fragment,cryopreserved,"0,1,2,3,4",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VHIO_PC-PDX-10,NOD SCID,NOD.CB-17-Prkdc <sup>scid</sup>/Rj,flank,heterotopic,tissue fragment,cryopreserved,"0,1,2,3,4,5,6,7,8,9",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14083,472M,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,Subcutaneous,heterotopic,tissue fragment,frozen,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14084,472M,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,Subcutaneous,heterotopic,tissue fragment,frozen,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14085,472M_Cx,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,Subcutaneous,heterotopic,tissue fragment,frozen,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14086,472M_Cx,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,Subcutaneous,heterotopic,tissue fragment,frozen,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
new_df = metadata[[
 'host_strain_name',
 'host_strain_nomenclature',
 'engraftment_site',
 'engraftment_type',
 'sample_type',
 'sample_state',
 'passage_number',
 'publications',
 'supplier',
 'supplier_type',
 'catalog_number',
 'vendor_link',
 'external_ids',
 'validation_technique',
 'description',
 'passages_tested',
 'validation_host_strain_nomenclature',
 'morphological_features',
 'SNP_analysis',
 'STR_analysis',
 'tumour_status',
 'model_purity',
 'comments',
 'collection_date',
 'collection_event',
 'months_since_collection_1',
 'age_in_years_at_collection',
 'diagnosis',
 'tumour_type',
 'primary_site',
 'collection_site',
 'collection_method',
 'stage',
 'staging_system',
 'grade',
 'grading_system',
 'virology_status',
 'gene_mutation_status',
 'sharable',
 'treatment_naive_at_collection',
 'treated_at_collection',
 'treated_prior_to_collection',
 'response_to_treatment',
 'sex',
 'history',
 'ethnicity',
 'ethnicity_assessment_method',
 'initial_diagnosis',
 'age_at_initial_diagnosis',
 'age_category',
 'smoking_status',
 'alcohol_status',
 'alcohol_frequency',
 'family_history_of_cancer']]

In [166]:
import pandas as pd


# Get unique values for each column
unique_cols = {col: new_df[col].str.lower().dropna().unique() for col in new_df.columns}

# Find the max number of unique values among all columns
max_len = max(len(v) for v in unique_cols.values())

# Pad each column's unique values with NaN to align them vertically
aligned_unique_df = pd.DataFrame({
    col: pd.Series(vals).reindex(range(max_len))
    for col, vals in unique_cols.items()
})

print(aligned_unique_df)


     host_strain_name                           host_strain_nomenclature  \
0            nod scid                 nod.cb-17-prkdc <sup>scid</sup>/rj   
1                 nsg  nod.cg-prkdc<sup>scid</sup> il2rg<sup>tm1wjl</...   
2          swiss nude                    nod.cb17-prkdc<sup>scid</sup>/j   
3                 nog                  nod.cb17-prkdc<sup>scid</sup>/szj   
4           nmri nude                     crl:nu(ico)-foxn1<sup>nu</sup>   
...               ...                                                ...   
1554              NaN                                                NaN   
1555              NaN                                                NaN   
1556              NaN                                                NaN   
1557              NaN                                                NaN   
1558              NaN                                                NaN   

                           engraftment_site engraftment_type  \
0                      

In [168]:
aligned_unique_df[[ 'sex', 'age_in_years_at_collection', 'diagnosis', 'tumour_type', 'primary_site', 'collection_site', 'sharable', 'treatment_naive_at_collection', 'host_strain_name', 'host_strain_nomenclature', 'engraftment_site', 'engraftment_type', 'sample_type', 'passage_number', 'supplier', 'supplier_type', 'catalog_number', 'vendor_link', 'validation_technique', 'description', 'passages_tested', 'validation_host_strain_nomenclature', 'SNP_analysis', 'STR_analysis', 'comments']].fillna('')

,sex,age_in_years_at_collection,diagnosis,tumour_type,primary_site,collection_site,sharable,treatment_naive_at_collection,host_strain_name,host_strain_nomenclature,...,supplier_type,catalog_number,vendor_link,validation_technique,description,passages_tested,validation_host_strain_nomenclature,SNP_analysis,STR_analysis,comments
0,female,not provided,pancreatic carcinoma - infiltrating ductal ade...,primary,pancreas,pancreas,not provided,not provided,nod scid,nod.cb-17-prkdc <sup>scid</sup>/rj,...,not provided,not provided,not provided,immunohistochemistry,not provided,not provided,nod.cb-17-prkdc <sup>scid</sup>/rj,not provided,not provided,not provided
1,male,69,pancreatic carcinoma - infiltrating ductal ade...,metastatic,bile duct,liver,yes,yes,nsg,nod.cg-prkdc<sup>scid</sup> il2rg<sup>tm1wjl</...,...,academic,gc0015_nsg_2,https://www.cccells.org/rclp.php,"immunohystochemistry of mouse spleen, flow cyt...",good concordance between xenograft and patient...,0,nod.cg-prkdc<sup>scid</sup> il2rg<sup>tm1wjl</...,no,no,the childhood cancer repository has a matching...
2,not provided,74,pancreatic carcinoma - intraductal papillary m...,not provided,peripheral blood,peritoneum,no,no,swiss nude,nod.cb17-prkdc<sup>scid</sup>/j,...,academia,cm18_nsg_345,,"immunohystochemistry of mouse spleen, flow cyt...",concordance between xenograft and patient tumo...,1,nod.cb17-prkdc<sup>scid</sup>/j,,yes,the childhood cancer repository has a matching...
3,not collected,59,pancreatic carcinoma,recurrent,breast,lung,,not collected,nog,nod.cb17-prkdc<sup>scid</sup>/szj,...,,pm129_nsg_6,,immunohystochemistry of human bc grown in mouse,some derivation between xenograft and patien...,6,nod.cb17-prkdc<sup>scid</sup>/szj,,,the childhood cancer repository has a matching...
4,other,54,cholangiocarcinoma,not collected,spleen/bone marrow,not provided,,no,nmri nude,crl:nu(ico)-foxn1<sup>nu</sup>,...,,breast_hsm_bc004.1_pdx,,not provided,high concordance between xenograft and patien...,3,crl:nu(ico)-foxn1<sup>nu</sup>,,,the childhood cancer repository has a matching...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,,,,,,,,,,,...,,463.45a,,,,,,,,
1555,,,,,,,,,,,...,,463.55a,,,,,,,,
1556,,,,,,,,,,,...,,463.58a,,,,,,,,
1557,,,,,,,,,,,...,,472m,,,,,,,,


In [170]:
aligned_unique_df.fillna('')

,host_strain_name,host_strain_nomenclature,engraftment_site,engraftment_type,sample_type,sample_state,passage_number,publications,supplier,supplier_type,...,history,ethnicity,ethnicity_assessment_method,initial_diagnosis,age_at_initial_diagnosis,age_category,smoking_status,alcohol_status,alcohol_frequency,family_history_of_cancer
0,nod scid,nod.cb-17-prkdc <sup>scid</sup>/rj,flank,heterotopic,tissue fragment,cryopreserved,"0,1,2,3,4,5",pmid: 28581516,not provided,not provided,...,not provided,not provided,not provided,pancreatic carcinoma - infiltrating ductal ade...,not provided,not provided,not provided,not provided,not provided,not provided
1,nsg,nod.cg-prkdc<sup>scid</sup> il2rg<sup>tm1wjl</...,peripheral blood,orthotopic,cell suspension,ffpe tissue mouse spleen,"1,2,3",pmid: 29074954,hsm,academic,...,"hereditary idiopathic hemochromatosis, alcohol...",italian,self-assessed,pancreatic carcinoma - infiltrating ductal ade...,49,adult,ex-smoker,yes,not collected,yes
2,swiss nude,nod.cb17-prkdc<sup>scid</sup>/j,mammary fat pad,not provided,bone marrow,ffpe,"0,1,2,3,4,5,6,7",pmid: 35819446,css,academia,...,"arterial hypertension, type 2 diabetes, hyperl...",western european,genetic,pancreatic carcinoma - intraductal papillary m...,70,pediatric,non-smoker,no,,no
3,nog,nod.cb17-prkdc<sup>scid</sup>/szj,bone marrow and spleen,subcutaneous,"bone marrow aspirates,peripheral blood",fresh,"0,1,2,3,4",pmid: 27815673,hci-bcm,,...,"arterial hypertension, type 2 diabetes, obesit...",mixed or unknown,not collected,pancreatic carcinoma,55,,current smoker,not collected,,"family history: 1st degree relative, lung cancer"
4,nmri nude,crl:nu(ico)-foxn1<sup>nu</sup>,"intravenous, either lateral tail vein",intravenous,bone biospy,frozen,"0,1,2,3,4,5,6,7,8,9",pmid: 31693904,nki,,...,"arterial hypertension, type 2 diabetes, obesit...",european,,cholangiocarcinoma,54,,non smoker,,,"family history: 1st degree relative, lung ca; ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,,,,,,,,,,,...,,,,,,,,,,
1555,,,,,,,,,,,...,,,,,,,,,,
1556,,,,,,,,,,,...,,,,,,,,,,
1557,,,,,,,,,,,...,,,,,,,,,,


In [161]:
pd.read_json('https://www.cancermodels.org/api/search_index?data_source=eq.WUSTL&select=external_model_id,histology')

,external_model_id,histology
0,WU-0014-T1,Colon Adenocarcinoma
1,WUSTL WHIM14,Basal-Like Breast Carcinoma
2,WUSTL SHIM9,Synovial Sarcoma
3,WUSTL 369-06,Colorectal Adenocarcinoma
4,WUSTL WHIM30,Basal-Like Breast Carcinoma
...,...,...
295,WUSTL 655-08,Colorectal Adenocarcinoma
296,WU-0075,Urothelial Carcinoma
297,WUSTL 268-06,Colorectal Adenocarcinoma
298,WU-0255-T1,Lung Adenocarcinoma
